## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Dec. 30th Jan. 6th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,December-31-2024,Condo/Co-op,2900 NW 42nd Ave Unit A510,Coconut Creek,FL,33066.0,250000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Coconut-Creek/2900-N...,MARMLS,A11701839,N,Y,26.264337,-80.183081


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df2 = df[(df['SOLD DATE'] > '2024-12-29') & (df['SOLD DATE'] < '2025-01-07')]

In [10]:
df2.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
227,PAST SALE,2025-01-06,Condo/Co-op,8730 N Sherman Cir #303,Miramar,FL,33025.0,220000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miramar/8730-N-Sherm...,MARMLS,A11559241,N,Y,25.991785,-80.260961
54,PAST SALE,2025-01-06,Condo/Co-op,710 N Ocean Blvd #911,Pompano Beach,FL,33062.0,330000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/710-N-...,Beaches MLS,F10473712,N,Y,26.243380,-80.087389
200,PAST SALE,2025-01-06,Condo/Co-op,340 Sunset Dr #1407,Fort Lauderdale,FL,33301.0,325000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/340-...,Beaches MLS,F10448220,N,Y,26.119178,-80.113365
189,PAST SALE,2025-01-06,Condo/Co-op,1865 S Ocean Dr Unit 14j,Hallandale,FL,33009.0,359000.0,1.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/186...,Beaches MLS,F10456139,N,Y,25.983740,-80.120430
177,PAST SALE,2025-01-06,Condo/Co-op,3515 Merrick Ln #1302,Margate,FL,33063.0,375000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Margate/3515-Merrick...,Beaches MLS,RX-11016950,N,Y,26.269439,-80.200403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,PAST SALE,2024-12-30,Condo/Co-op,4800 Bayview Dr #605,Fort Lauderdale,FL,33308.0,365000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/4800...,Beaches MLS,F10421492,N,Y,26.188293,-80.106631
112,PAST SALE,2024-12-30,Condo/Co-op,4031 Lyndhurst J #4031,Deerfield Beach,FL,33442.0,145000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Deerfield-Beach/4031...,Beaches MLS,F10462564,N,Y,26.309212,-80.135940
172,PAST SALE,2024-12-30,Condo/Co-op,4520 El Mar Dr #31,Fort Lauderdale,FL,33308.0,355000.0,0.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-By-The-Se...,Beaches MLS,F10458138,N,Y,26.193530,-80.095072
83,PAST SALE,2024-12-30,Condo/Co-op,3100 N Ocean Blvd #2205,Fort Lauderdale,FL,33308.0,1200000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Beaches MLS,F10439512,N,Y,26.168258,-80.099279


In [11]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [12]:
df_top_ten = df2.head(10)

## Collect Agent Information

In [13]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [14]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [15]:
response_df = pd.DataFrame(response_list)

In [16]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [17]:
df_top_ten = merged_df

## Current Week's Values

In [18]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

100


In [19]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$344,588


In [20]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$271


In [21]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$34,458,824


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# Define the date range
start_date = '2024-12-23'
end_date = '2024-12-29'

# Filter the DataFrame between two dates (inclusive)
old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [23]:
# comparison week total sales
print(f'Comparison week filtered sales: {(len(old_week))}')
print('-----------')
print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
print('-----------')
print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
print('-----------')
print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
print('-----------')

Comparison week filtered sales: 87
-----------
Comparison week average sales price: $267,836.20
-----------
Comparison week average psf: $225.33
-----------
Comparison week sales total: $23,301,749.00
-----------


In [24]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Broward
Input Previous Week Condo Sales Total (number units sold):
87
Input Previous Week Condo Average Sales Price:
267836
Input Previous Week Condo Average PSF:
225
Input Previous Week Condo Sales total (ex: 198_000_000)
23301749


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [25]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [26]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [27]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [28]:
pd.set_option('display.max_columns',None)

In [29]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-03,Condo/Co-op,1 N Fort Lauderdale Beach Blvd #2002,Fort Lauderdale,FL,33304.0,2800000.0,2.0,2.5,Ritz Carlton,2083.0,NaN,2007.0,NaN,1344.0,3335.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1-N-...,Beaches MLS,F10461161,N,Y,26.123021,-80.104776,https://www.redfin.com/FL/Fort-Lauderdale/1-N-...,Cristian Michaels,Douglas Elliman,Daniel Teixeira,Douglas Elliman,Howard Elfman,The Agency Florida LLC,None,None,1,orange


In [30]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [31]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-03,Condo/Co-op,1 N Fort Lauderdale Beach Blvd #2002,Fort Lauderdale,FL,33304.0,2800000.0,2.0,2.5,Ritz Carlton,2083.0,NaN,2007.0,NaN,1344.0,3335.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1-N-...,Beaches MLS,F10461161,N,Y,26.123021,-80.104776,https://www.redfin.com/FL/Fort-Lauderdale/1-N-...,Cristian Michaels,Douglas Elliman,Daniel Teixeira,Douglas Elliman,Howard Elfman,The Agency Florida LLC,None,None,1,orange
1,PAST SALE,2025-01-02,Condo/Co-op,100 E Las Olas Blvd #3303,Fort Lauderdale,FL,33301.0,2100000.0,4.0,3.5,100 Las Olas Condo,2964.0,NaN,2020.0,NaN,709.0,3387.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Beaches MLS,RX-11010066,N,Y,26.118835,-80.141476,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Amy Best,Best Realty LLC,None,None,Kim Lekas,"Premier Estate Properties, Inc",None,None,2,blue
2,PAST SALE,2024-12-30,Condo/Co-op,3200 N Ocean Blvd #507,Fort Lauderdale,FL,33308.0,2100000.0,3.0,2.5,L'hermitage Ii,2332.0,NaN,1998.0,NaN,901.0,2955.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Beaches MLS,F10470398,N,Y,26.168793,-80.098465,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Patricia Post,"Post and Post, Inc.",None,None,David Marder,"Compass Florida, LLC",None,"Compass Florida, LLC",3,blue
3,PAST SALE,2024-12-30,Condo/Co-op,333 Las Olas Way #3906,Fort Lauderdale,FL,33301.0,2075000.0,3.0,3.5,Las Olas River House,3189.0,NaN,2004.0,NaN,651.0,3625.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/333-...,Beaches MLS,F10449636,N,Y,26.118354,-80.141518,https://www.redfin.com/FL/Fort-Lauderdale/333-...,Joshua Dotoli,"Compass Florida, LLC",Dayana Susterman,"Compass Florida, LLC",Glen Primak,The Agency Florida LLC,None,None,4,blue
4,PAST SALE,2024-12-30,Condo/Co-op,1700 S Ocean Blvd Unit 14A,Lauderdale By The Sea,FL,33062.0,1825000.0,3.0,3.0,Cristelle,3200.0,NaN,1998.0,NaN,570.0,2627.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1700-S...,Beaches MLS,F10452246,N,Y,26.209206,-80.092779,https://www.redfin.com/FL/Pompano-Beach/1700-S...,David Saindon,RE/MAX Experience,Robert Shastany,RE/MAX Experience,David Saindon,RE/MAX Experience,Robert Shastany,RE/MAX Experience,5,blue
5,PAST SALE,2024-12-30,Condo/Co-op,1230 Hillsboro Mile #107,Hillsboro Beach,FL,33062.0,1690000.0,3.0,3.0,Mediterranea Condo,2730.0,NaN,1999.0,NaN,619.0,2491.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hillsboro-Beach/1230...,Beaches MLS,F10473227,N,Y,26.304206,-80.077857,https://www.redfin.com/FL/Hillsboro-Beach/1230...,Kun Li,Rainbow Investment Realty Inc,None,None,Anthony Albino,Beautiful Homes Realty Inc,None,None,6,blue
6,PAST SALE,2024-12-30,Condo/Co-op,3100 N Ocean Blvd #2205,Fort Lauderdale,FL,33308.0,1200000.0,2.0,2.0,L'hermitage,1537.0,NaN,1997.0,NaN,781.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Beaches MLS,F10439512,N,Y,26.168258,-80.099279,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Evie Fortney,"Compass Florida, LLC",None,None,Patricia Post,"Post and Post, Inc.",David M Manupella,"Post and Post, Inc.",7,blue
7,PAST SALE,2024-12-30,Condo/Co-op,4300 N Ocean Blvd Unit 7J,Fort Lauderdale,FL,33308.0,829000.0,2.0,2.0,Plaza East,1450.0,NaN,1967.0,NaN,572.0,1617.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/4300...,Beaches MLS,F10464021,N,Y,26.181938,-80.096840,https://www.redfin.com/FL/Fort-Lauderdale/4300...,Lilli Schipper,LoKation,None,None,Leonid Belfor,"One Way Realty Group, LLC",None,None,8,blue
8,PAST SALE,2024-12-31,Condo/Co-op,1201 S Ocean Dr Unit 1201N,Hollywood,FL,3301

In [37]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [38]:
m.save('index.html')

## Data snagger

In [39]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [40]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-03,Condo/Co-op,1 N Fort Lauderdale Beach Blvd #2002,Fort Lauderdale,FL,33304.0,2800000.0,2.0,2.5,Ritz Carlton,2083.0,NaN,2007.0,NaN,1344.0,3335.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1-N-...,Beaches MLS,F10461161,N,Y,26.123021,-80.104776,https://www.redfin.com/FL/Fort-Lauderdale/1-N-...,Cristian Michaels,Douglas Elliman,Daniel Teixeira,Douglas Elliman,Howard Elfman,The Agency Florida LLC,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [41]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Broward_County_condo_sales_week_ending_01062024


# CREATE TEMPLATE 

In [42]:
muni_set = set(df_top_ten['CITY'])

In [43]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [44]:
df_top_ten.reset_index(inplace=True,drop=True)

In [45]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [46]:
top_sale

'Ritz Carlton at 1 N Fort Lauderdale Beach Blvd #2002 in Fort Lauderdale'

In [47]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [48]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [49]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-03,Condo/Co-op,1 N Fort Lauderdale Beach Blvd #2002,Fort Lauderdale,FL,33304.0,2800000.0,2.0,2.5,Ritz Carlton,2083.0,NaN,2007.0,NaN,1344.0,3335.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1-N-...,Beaches MLS,F10461161,N,Y,26.123021,-80.104776,https://www.redfin.com/FL/Fort-Lauderdale/1-N-...,Cristian Michaels,Douglas Elliman,Daniel Teixeira,Douglas Elliman,Howard Elfman,The Agency Florida LLC,None,None,1,orange,Ritz Carlton at 1 N Fort Lauderdale Beach Blvd...
1,PAST SALE,2025-01-02,Condo/Co-op,100 E Las Olas Blvd #3303,Fort Lauderdale,FL,33301.0,2100000.0,4.0,3.5,100 Las Olas Condo,2964.0,NaN,2020.0,NaN,709.0,3387.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Beaches MLS,RX-11010066,N,Y,26.118835,-80.141476,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Amy Best,Best Realty LLC,None,None,Kim Lekas,"Premier Estate Properties, Inc",None,None,2,blue,100 Las Olas Condo at 100 E Las Olas Blvd #330...
2,PAST SALE,2024-12-30,Condo/Co-op,3200 N Ocean Blvd #507,Fort Lauderdale,FL,33308.0,2100000.0,3.0,2.5,L'hermitage Ii,2332.0,NaN,1998.0,NaN,901.0,2955.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Beaches MLS,F10470398,N,Y,26.168793,-80.098465,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Patricia Post,"Post and Post, Inc.",None,None,David Marder,"Compass Florida, LLC",None,"Compass Florida, LLC",3,blue,L'hermitage Ii at 3200 N Ocean Blvd #507 in Fo...
3,PAST SALE,2024-12-30,Condo/Co-op,333 Las Olas Way #3906,Fort Lauderdale,FL,33301.0,2075000.0,3.0,3.5,Las Olas River House,3189.0,NaN,2004.0,NaN,651.0,3625.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/333-...,Beaches MLS,F10449636,N,Y,26.118354,-80.141518,https://www.redfin.com/FL/Fort-Lauderdale/333-...,Joshua Dotoli,"Compass Florida, LLC",Dayana Susterman,"Compass Florida, LLC",Glen Primak,The Agency Florida LLC,None,None,4,blue,Las Olas River House at 333 Las Olas Way #3906...
4,PAST SALE,2024-12-30,Condo/Co-op,1700 S Ocean Blvd Unit 14A,Lauderdale By The Sea,FL,33062.0,1825000.0,3.0,3.0,Cristelle,3200.0,NaN,1998.0,NaN,570.0,2627.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1700-S...,Beaches MLS,F10452246,N,Y,26.209206,-80.092779,https://www.redfin.com/FL/Pompano-Beach/1700-S...,David Saindon,RE/MAX Experience,Robert Shastany,RE/MAX Experience,David Saindon,RE/MAX Experience,Robert Shastany,RE/MAX Experience,5,blue,Cristelle at 1700 S Ocean Blvd Unit 14A in Lau...
5,PAST SALE,2024-12-30,Condo/Co-op,1230 Hillsboro Mile #107,Hillsboro Beach,FL,33062.0,1690000.0,3.0,3.0,Mediterranea Condo,2730.0,NaN,1999.0,NaN,619.0,2491.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hillsboro-Beach/1230...,Beaches MLS,F10473227,N,Y,26.304206,-80.077857,https://www.redfin.com/FL/Hillsboro-Beach/1230...,Kun Li,Rainbow Investment Realty Inc,None,None,Anthony Albino,Beautiful Homes Realty Inc,None,None,6,blue,Mediterranea Condo at 1230 Hillsboro Mile #107...
6,PAST SALE,2024-12-30,Condo/Co-op,3100 N Ocean Blvd #2205,Fort Lauderdale,FL,33308.0,1200000.0,2.0,2.0,L'hermitage,1537.0,NaN,1997.0,NaN,781.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Beaches MLS,F10439512,N,Y,26.168258,-80.099279,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Evie Fortney,"Compass Florida, LLC",None,None,Patricia Post,"Post and Post, Inc.",David M Manupella,"Post and Post, Inc.",7,blue,L'hermitage at 3100 N Ocean Blvd #2205 in Fort...
7,PAST SALE,2024-12-30,Condo/Co-op,4300 N Ocean Blvd Unit 7J,Fort Lauderdale,FL,33308.0,829000.0,2.0,2.0,Plaza East,1450.0,

In [50]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Ritz Carlton closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $570,000 to $2,800,000
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Broward condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Pompano Beach, Hollywood, Lauderdale By The Sea, Hillsboro Beach, Fort Lauderdale

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 100 condo sales totaling $34,458,824 million from Dec. 30th to Jan. 6th. The previous week, brokers closed 87 condo sales totaling $23,301,749.

Last week’s units sold for an average of $344,588, higher than the $267,836 average price from the previous week. The a

In [51]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [52]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [53]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [54]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-03,Condo/Co-op,1 N Fort Lauderdale Beach Blvd #2002,Fort Lauderdale,FL,33304.0,2800000.0,2.0,2.5,Ritz Carlton,2083.0,NaN,2007.0,NaN,1344.0,3335.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1-N-...,Beaches MLS,F10461161,N,Y,26.123021,-80.104776,https://www.redfin.com/FL/Fort-Lauderdale/1-N-...,Cristian Michaels,Douglas Elliman,Daniel Teixeira,Douglas Elliman,Howard Elfman,The Agency Florida LLC,None,None,1,orange,Ritz Carlton at 1 N Fort Lauderdale Beach Blvd...
1,PAST SALE,2025-01-02,Condo/Co-op,100 E Las Olas Blvd #3303,Fort Lauderdale,FL,33301.0,2100000.0,4.0,3.5,100 Las Olas Condo,2964.0,NaN,2020.0,NaN,709.0,3387.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Beaches MLS,RX-11010066,N,Y,26.118835,-80.141476,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Amy Best,Best Realty LLC,None,None,Kim Lekas,"Premier Estate Properties, Inc",None,None,2,blue,100 Las Olas Condo at 100 E Las Olas Blvd #330...
2,PAST SALE,2024-12-30,Condo/Co-op,3200 N Ocean Blvd #507,Fort Lauderdale,FL,33308.0,2100000.0,3.0,2.5,L'hermitage Ii,2332.0,NaN,1998.0,NaN,901.0,2955.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Beaches MLS,F10470398,N,Y,26.168793,-80.098465,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Patricia Post,"Post and Post, Inc.",None,None,David Marder,"Compass Florida, LLC",None,"Compass Florida, LLC",3,blue,L'hermitage Ii at 3200 N Ocean Blvd #507 in Fo...
3,PAST SALE,2024-12-30,Condo/Co-op,333 Las Olas Way #3906,Fort Lauderdale,FL,33301.0,2075000.0,3.0,3.5,Las Olas River House,3189.0,NaN,2004.0,NaN,651.0,3625.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/333-...,Beaches MLS,F10449636,N,Y,26.118354,-80.141518,https://www.redfin.com/FL/Fort-Lauderdale/333-...,Joshua Dotoli,"Compass Florida, LLC",Dayana Susterman,"Compass Florida, LLC",Glen Primak,The Agency Florida LLC,None,None,4,blue,Las Olas River House at 333 Las Olas Way #3906...
4,PAST SALE,2024-12-30,Condo/Co-op,1700 S Ocean Blvd Unit 14A,Lauderdale By The Sea,FL,33062.0,1825000.0,3.0,3.0,Cristelle,3200.0,NaN,1998.0,NaN,570.0,2627.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1700-S...,Beaches MLS,F10452246,N,Y,26.209206,-80.092779,https://www.redfin.com/FL/Pompano-Beach/1700-S...,David Saindon,RE/MAX Experience,Robert Shastany,RE/MAX Experience,David Saindon,RE/MAX Experience,Robert Shastany,RE/MAX Experience,5,blue,Cristelle at 1700 S Ocean Blvd Unit 14A in Lau...
5,PAST SALE,2024-12-30,Condo/Co-op,1230 Hillsboro Mile #107,Hillsboro Beach,FL,33062.0,1690000.0,3.0,3.0,Mediterranea Condo,2730.0,NaN,1999.0,NaN,619.0,2491.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hillsboro-Beach/1230...,Beaches MLS,F10473227,N,Y,26.304206,-80.077857,https://www.redfin.com/FL/Hillsboro-Beach/1230...,Kun Li,Rainbow Investment Realty Inc,None,None,Anthony Albino,Beautiful Homes Realty Inc,None,None,6,blue,Mediterranea Condo at 1230 Hillsboro Mile #107...
6,PAST SALE,2024-12-30,Condo/Co-op,3100 N Ocean Blvd #2205,Fort Lauderdale,FL,33308.0,1200000.0,2.0,2.0,L'hermitage,1537.0,NaN,1997.0,NaN,781.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Beaches MLS,F10439512,N,Y,26.168258,-80.099279,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Evie Fortney,"Compass Florida, LLC",None,None,Patricia Post,"Post and Post, Inc.",David M Manupella,"Post and Post, Inc.",7,blue,L'hermitage at 3100 N Ocean Blvd #2205 in Fort...
7,PAST SALE,2024-12-30,Condo/Co-op,4300 N Ocean Blvd Unit 7J,Fort Lauderdale,FL,33308.0,829000.0,2.0,2.0,Plaza East,1450.0,

In [55]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: Ritz Carlton closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $570,000 to $2,800,000
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Broward condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Pompano Beach, Hollywood, Lauderdale By The Sea, Hillsboro Beach, Fort Lauderdale

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 100 condo sales totaling $34,458,824 million from Dec. 30th to Jan. 6th. The previous week, brokers closed 87 condo sales totaling $23,301,749.

Last week’s units sold for an average of $344,588, higher than the $267,836 average price from the previous week. The a

In [56]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Fort-Lauderdale/1-N-Fort-Lauderdale-Beach-Blvd-33304/unit-2002/home/42073399


In [57]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Fort-Lauderdale/100-E-Las-Olas-Blvd-33301/unit-3303/home/148167387


In [58]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Hillsboro-Beach/1051-Hillsboro-Mile-33062/unit-305E/home/41517338


In [59]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Hollywood/1201-S-Ocean-Dr-33019/unit-1201N/home/41938650


In [60]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Fort-Lauderdale/1-N-Fort-Lauderdale-Beach-Blvd-33304/unit-2002/home/42073399


In [61]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Hollywood/1201-S-Ocean-Dr-33019/unit-1201N/home/41938650


In [62]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-03,Condo/Co-op,1 N Fort Lauderdale Beach Blvd #2002,Fort Lauderdale,FL,33304.0,2800000.0,2.0,2.5,Ritz Carlton,2083.0,NaN,2007.0,NaN,1344.0,3335.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1-N-...,Beaches MLS,F10461161,N,Y,26.123021,-80.104776,https://www.redfin.com/FL/Fort-Lauderdale/1-N-...,Cristian Michaels,Douglas Elliman,Daniel Teixeira,Douglas Elliman,Howard Elfman,The Agency Florida LLC,None,None,1,orange,Ritz Carlton at 1 N Fort Lauderdale Beach Blvd...
1,PAST SALE,2025-01-02,Condo/Co-op,100 E Las Olas Blvd #3303,Fort Lauderdale,FL,33301.0,2100000.0,4.0,3.5,100 Las Olas Condo,2964.0,NaN,2020.0,NaN,709.0,3387.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Beaches MLS,RX-11010066,N,Y,26.118835,-80.141476,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Amy Best,Best Realty LLC,None,None,Kim Lekas,"Premier Estate Properties, Inc",None,None,2,blue,100 Las Olas Condo at 100 E Las Olas Blvd #330...
2,PAST SALE,2024-12-30,Condo/Co-op,3200 N Ocean Blvd #507,Fort Lauderdale,FL,33308.0,2100000.0,3.0,2.5,L'hermitage Ii,2332.0,NaN,1998.0,NaN,901.0,2955.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Beaches MLS,F10470398,N,Y,26.168793,-80.098465,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Patricia Post,"Post and Post, Inc.",None,None,David Marder,"Compass Florida, LLC",None,"Compass Florida, LLC",3,blue,L'hermitage Ii at 3200 N Ocean Blvd #507 in Fo...
3,PAST SALE,2024-12-30,Condo/Co-op,333 Las Olas Way #3906,Fort Lauderdale,FL,33301.0,2075000.0,3.0,3.5,Las Olas River House,3189.0,NaN,2004.0,NaN,651.0,3625.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/333-...,Beaches MLS,F10449636,N,Y,26.118354,-80.141518,https://www.redfin.com/FL/Fort-Lauderdale/333-...,Joshua Dotoli,"Compass Florida, LLC",Dayana Susterman,"Compass Florida, LLC",Glen Primak,The Agency Florida LLC,None,None,4,blue,Las Olas River House at 333 Las Olas Way #3906...
4,PAST SALE,2024-12-30,Condo/Co-op,1700 S Ocean Blvd Unit 14A,Lauderdale By The Sea,FL,33062.0,1825000.0,3.0,3.0,Cristelle,3200.0,NaN,1998.0,NaN,570.0,2627.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1700-S...,Beaches MLS,F10452246,N,Y,26.209206,-80.092779,https://www.redfin.com/FL/Pompano-Beach/1700-S...,David Saindon,RE/MAX Experience,Robert Shastany,RE/MAX Experience,David Saindon,RE/MAX Experience,Robert Shastany,RE/MAX Experience,5,blue,Cristelle at 1700 S Ocean Blvd Unit 14A in Lau...
5,PAST SALE,2024-12-30,Condo/Co-op,1230 Hillsboro Mile #107,Hillsboro Beach,FL,33062.0,1690000.0,3.0,3.0,Mediterranea Condo,2730.0,NaN,1999.0,NaN,619.0,2491.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hillsboro-Beach/1230...,Beaches MLS,F10473227,N,Y,26.304206,-80.077857,https://www.redfin.com/FL/Hillsboro-Beach/1230...,Kun Li,Rainbow Investment Realty Inc,None,None,Anthony Albino,Beautiful Homes Realty Inc,None,None,6,blue,Mediterranea Condo at 1230 Hillsboro Mile #107...
6,PAST SALE,2024-12-30,Condo/Co-op,3100 N Ocean Blvd #2205,Fort Lauderdale,FL,33308.0,1200000.0,2.0,2.0,L'hermitage,1537.0,NaN,1997.0,NaN,781.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Beaches MLS,F10439512,N,Y,26.168258,-80.099279,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Evie Fortney,"Compass Florida, LLC",None,None,Patricia Post,"Post and Post, Inc.",David M Manupella,"Post and Post, Inc.",7,blue,L'hermitage at 3100 N Ocean Blvd #2205 in Fort...
7,PAST SALE,2024-12-30,Condo/Co-op,4300 N Ocean Blvd Unit 7J,Fort Lauderdale,FL,33308.0,829000.0,2.0,2.0,Plaza East,1450.0,

## Time on Market Calculator

In [64]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 8, 5) ## List (Earlier) date
date2 = datetime(2025, 1, 2) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

150


## Clean CSV for Datawrapper Chart

In [65]:
chart_df = df_top_ten

In [66]:
chart_df = chart_df.fillna(" ")

In [67]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [68]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [69]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [70]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [71]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [72]:
csv_date_string = today.strftime("%m_%d_%Y")

In [73]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [74]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-01-03,1 N Fort Lauderdale Beach Blvd #2002 in Fort L...,"$2,800,000",2,2.5,"2,083",2007,"$1,344",Beaches MLS,F10461161,Cristian Michaels Douglas Elliman Daniel Teixe...,Howard Elfman The Agency Florida LLC
1,2025-01-02,100 E Las Olas Blvd #3303 in Fort Lauderdale,"$2,100,000",4,3.5,"2,964",2020,$709,Beaches MLS,RX-11010066,Amy Best Best Realty LLC,"Kim Lekas Premier Estate Properties, Inc"
2,2024-12-30,3200 N Ocean Blvd #507 in Fort Lauderdale,"$2,100,000",3,2.5,"2,332",1998,$901,Beaches MLS,F10470398,"Patricia Post Post and Post, Inc.","David Marder Compass Florida, LLC Compass Fl..."
3,2024-12-30,333 Las Olas Way #3906 in Fort Lauderdale,"$2,075,000",3,3.5,"3,189",2004,$651,Beaches MLS,F10449636,"Joshua Dotoli Compass Florida, LLC Dayana Sust...",Glen Primak The Agency Florida LLC
4,2024-12-30,1700 S Ocean Blvd Unit 14A in Lauderdale By Th...,"$1,825,000",3,3.0,"3,200",1998,$570,Beaches MLS,F10452246,David Saindon RE/MAX Experience Robert Shastan...,David Saindon RE/MAX Experience Robert Shastan...
5,2024-12-30,1230 Hillsboro Mile #107 in Hillsboro Beach,"$1,690,000",3,3.0,"2,730",1999,$619,Beaches MLS,F10473227,Kun Li Rainbow Investment Realty Inc,Anthony Albino Beautiful Homes Realty Inc
6,2024-12-30,3100 N Ocean Blvd #2205 in Fort Lauderdale,"$1,200,000",2,2.0,"1,537",1997,$781,Beaches MLS,F10439512,"Evie Fortney Compass Florida, LLC","Patricia Post Post and Post, Inc. David M Manu..."
7,2024-12-30,4300 N Ocean Blvd Unit 7J in Fort Lauderdale,"$829,000",2,2.0,"1,450",1967,$572,Beaches MLS,F10464021,Lilli Schipper LoKation,"Leonid Belfor One Way Realty Group, LLC"
8,2024-12-31,1201 S Ocean Dr Unit 1201N in Hollywood,"$820,000",3,2.5,"1,990",1982,$412,MARMLS,A11652129,Amy Greenberg Summit Real Estate Brokerage Mel...,Juliana Moraes Legacy Plus Realty Inc
9,2024-12-30,1051 Hillsboro Mile Unit 305 E in Pompano Beach,"$570,000",2,2.0,"1,000",1969,$570,Beaches MLS,F10465207,Lisa Guinta BlueChip Properties,Lisa Guinta BlueChip Properties
